In [186]:
# import code block
from selenium import webdriver
from bs4 import BeautifulSoup
import zipfile
import pandas as pd
import csv
import time
import random
import matplotlib.pyplot as plt
from urllib.request import urlopen
import re
import string
import numpy as np
import sys
# !{sys.executable} -m pip --upgrade pip
# !{sys.executable} -m pip install descartes
!{sys.executable} -m pip install geopandas
# import descartes
import geopandas as gpd
# from shapely.geometry import Point, Polygon
# from shapely import wkt
from datetime import date
import datetime
from statsmodels.tsa.arima.model import ARIMA
from random import random

In [123]:
evictionLab = pd.read_csv('tracts.csv')
evictionLab.rename(columns={'name':'CETRACT'}, inplace=True)
evictionLab

,GEOID,year,CETRACT,parent-location,population,poverty-rate,renter-occupied-households,pct-renter-occupied,median-gross-rent,median-household-income,...,pct-nh-pi,pct-multiple,pct-other,eviction-filings,evictions,eviction-rate,eviction-filing-rate,low-flag,imputed,subbed
0,25001010100,2000,101.0,"Barnstable County, Massachusetts",3433.09,16.28,849,46.2,644.01,32722.44,...,0.0,2.88,0.82,NaN,NaN,NaN,NaN,0,0,0
1,25001010100,2001,101.0,"Barnstable County, Massachusetts",3433.09,16.28,842,46.2,644.01,32722.44,...,0.0,2.88,0.82,NaN,NaN,NaN,NaN,0,0,0
2,25001010100,2002,101.0,"Barnstable County, Massachusetts",3433.09,16.28,836,46.2,644.01,32722.44,...,0.0,2.88,0.82,NaN,NaN,NaN,NaN,0,0,0
3,25001010100,2003,101.0,"Barnstable County, Massachusetts",3433.09,16.28,829,46.2,644.01,32722.44,...,0.0,2.88,0.82,31.0,30.0,3.62,3.74,0,0,0
4,25001010100,2004,101.0,"Barnstable County, Massachusetts",3433.09,16.28,822,46.2,644.01,32722.44,...,0.0,2.88,0.82,NaN,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25136,25027761400,2012,7614.0,"Worcester County, Massachusetts",6567.00,4.19,174,8.4,1333.00,126061.00,...,0.0,2.50,0.00,1.0,1.0,0.57,0.57,1,0,0
25137,25027761400,2013,7614.0,"Worcester County, Massachusetts",6567.00,4.19,180,8.4,1333.00,126061.00,...,0.0,2.50,0.00,4.0,4.0,2.23,2.23,1,0,0
25138,25027761400,2014,7614.0,"Worcester County, Massachusetts",6567.00,4.19,185,8.4,1333.00,126061.00,...,0.0,2.50,0.00,1.0,1.0,0.54,0.54,1,0,0
25139,25027761400,2015,7614.0,"Worcester County, Massachusetts",6567.00,4.19,191,8.4,1333.00,126061.00,...,0.0,2.50,0.00,0.0,0.0,0.00,0.00,0,0,0


In [133]:
forecasting_df = evictionLab[['CETRACT', 'year','evictions', 'eviction-rate']].copy()
list_of_cetracts = forecasting_df['CETRACT'].unique()

forecasting_df = forecasting_df.fillna(0)
forecasting_df = forecasting_df.reset_index()
forecasting_df = forecasting_df.drop(['index'], axis=1)

forecasting_df.head(20)

,CETRACT,year,evictions,eviction-rate
0,101.00,2000,0.0,0.00
1,101.00,2001,0.0,0.00
2,101.00,2002,0.0,0.00
3,101.00,2003,30.0,3.62
4,101.00,2004,0.0,0.00
5,101.00,2005,0.0,0.00
6,101.00,2006,0.0,0.00
7,101.00,2007,0.0,0.00
8,101.00,2008,0.0,0.00
9,101.00,2009,0.0,0.00


In [224]:
eviction_dict = {}

for index, row in forecasting_df.iterrows():
    if row['CETRACT'] not in eviction_dict:
        eviction_dict[row['CETRACT']] = [[row['year'], row['evictions'], row['eviction-rate']]]
    else:
        eviction_dict[row['CETRACT']].append([row['year'], row['evictions'], row['eviction-rate']])


In [225]:
# dictionary format is: Key == CETRACT, and each key maps to a 2D array where each list item is in the form of 
# [year, # of evictions, and eviction rate]
print(eviction_dict)

{101.0: [[2000.0, 0.0, 0.0], [2001.0, 0.0, 0.0], [2002.0, 0.0, 0.0], [2003.0, 30.0, 3.62], [2004.0, 0.0, 0.0], [2005.0, 0.0, 0.0], [2006.0, 0.0, 0.0], [2007.0, 0.0, 0.0], [2008.0, 0.0, 0.0], [2009.0, 0.0, 0.0], [2010.0, 0.0, 0.0], [2011.0, 0.0, 0.0], [2012.0, 0.0, 0.0], [2013.0, 0.0, 0.0], [2014.0, 6.0, 0.72], [2015.0, 11.0, 1.3], [2016.0, 6.0, 0.7]], 102.06: [[2000.0, 0.0, 0.0], [2001.0, 0.0, 0.0], [2002.0, 0.0, 0.0], [2003.0, 13.0, 4.39], [2004.0, 0.0, 0.0], [2005.0, 0.0, 0.0], [2006.0, 0.0, 0.0], [2007.0, 0.0, 0.0], [2008.0, 0.0, 0.0], [2009.0, 0.0, 0.0], [2010.0, 0.0, 0.0], [2011.0, 0.0, 0.0], [2012.0, 0.0, 0.0], [2013.0, 0.0, 0.0], [2014.0, 7.0, 2.26], [2015.0, 6.0, 1.89], [2016.0, 4.0, 1.23]], 102.08: [[2000.0, 0.0, 0.0], [2001.0, 0.0, 0.0], [2002.0, 0.0, 0.0], [2003.0, 8.0, 3.88], [2004.0, 0.0, 0.0], [2005.0, 0.0, 0.0], [2006.0, 0.0, 0.0], [2007.0, 0.0, 0.0], [2008.0, 0.0, 0.0], [2009.0, 0.0, 0.0], [2010.0, 0.0, 0.0], [2011.0, 0.0, 0.0], [2012.0, 0.0, 0.0], [2013.0, 0.0, 0.0], [

In [226]:
# forecasting for first key in dictionary AKA CETRACT == 101.00
# use moving average for forecasting
test_dict = eviction_dict[101.00]
test_dict

[[2000.0, 0.0, 0.0],
 [2001.0, 0.0, 0.0],
 [2002.0, 0.0, 0.0],
 [2003.0, 30.0, 3.62],
 [2004.0, 0.0, 0.0],
 [2005.0, 0.0, 0.0],
 [2006.0, 0.0, 0.0],
 [2007.0, 0.0, 0.0],
 [2008.0, 0.0, 0.0],
 [2009.0, 0.0, 0.0],
 [2010.0, 0.0, 0.0],
 [2011.0, 0.0, 0.0],
 [2012.0, 0.0, 0.0],
 [2013.0, 0.0, 0.0],
 [2014.0, 6.0, 0.72],
 [2015.0, 11.0, 1.3],
 [2016.0, 6.0, 0.7]]

In [238]:
# x is the list of data (i.e. evictions) and w is the window we want to use to predict the next value
# returns the average of the last w items, which will be used as the predicted value
def moving_average(x, w):
    sum = 0
    for x in x[-w:]:
        sum += x
    return sum//w 

def predict_from_moving_average(x):
    # contrived dataset
    data = x
    # fit model
    model = ARIMA(data, order=(0, 0, 1))
    model_fit = model.fit()
    # make prediction
    yhat = model_fit.predict(len(data), len(data))
    return yhat


years = []
evictions = []
eviction_rate = []

for i in range(len(test_dict)):
    years.append(int(test_dict[i][0]))
    evictions.append(int(test_dict[i][1]))
    eviction_rate.append(test_dict[i][2])


print(years)
print(evictions)
print(eviction_rate)

[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
[0, 0, 0, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 11, 6]
[0.0, 0.0, 0.0, 3.62, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.72, 1.3, 0.7]


In [240]:
# make predictions for next 4 years (to 2020)
for i in range(4):
    years.append(years[-1] + 1)
    evictions.append(int(predict_from_moving_average(evictions)))
    eviction_rate.append(float(predict_from_moving_average(eviction_rate)))
    
print(years)
print(evictions)
print(eviction_rate)

[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
[0, 0, 0, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 11, 6, 3, 3, 3, 3, 3]
[0.0, 0.0, 0.0, 3.62, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.72, 1.3, 0.7, 0.35990091861115675, 0.3729904766911589, 0.37299741844867246, 0.37299975634739685]


In [241]:
# update dictionary values
for i in range(17, 21):
    eviction_dict[list_of_cetracts[0]].append([years[i], evictions[i], eviction_rate[i]])
print(eviction_dict[list_of_cetracts[0]])

[[2000.0, 0.0, 0.0], [2001.0, 0.0, 0.0], [2002.0, 0.0, 0.0], [2003.0, 30.0, 3.62], [2004.0, 0.0, 0.0], [2005.0, 0.0, 0.0], [2006.0, 0.0, 0.0], [2007.0, 0.0, 0.0], [2008.0, 0.0, 0.0], [2009.0, 0.0, 0.0], [2010.0, 0.0, 0.0], [2011.0, 0.0, 0.0], [2012.0, 0.0, 0.0], [2013.0, 0.0, 0.0], [2014.0, 6.0, 0.72], [2015.0, 11.0, 1.3], [2016.0, 6.0, 0.7], [2017, 3, 0.35990091861115675], [2018, 3, 0.3729904766911589], [2019, 3, 0.37299741844867246], [2020, 3, 0.37299975634739685]]


In [242]:
eviction_dict

{101.0: [[2000.0, 0.0, 0.0],
  [2001.0, 0.0, 0.0],
  [2002.0, 0.0, 0.0],
  [2003.0, 30.0, 3.62],
  [2004.0, 0.0, 0.0],
  [2005.0, 0.0, 0.0],
  [2006.0, 0.0, 0.0],
  [2007.0, 0.0, 0.0],
  [2008.0, 0.0, 0.0],
  [2009.0, 0.0, 0.0],
  [2010.0, 0.0, 0.0],
  [2011.0, 0.0, 0.0],
  [2012.0, 0.0, 0.0],
  [2013.0, 0.0, 0.0],
  [2014.0, 6.0, 0.72],
  [2015.0, 11.0, 1.3],
  [2016.0, 6.0, 0.7],
  [2017, 3, 0.35990091861115675],
  [2018, 3, 0.3729904766911589],
  [2019, 3, 0.37299741844867246],
  [2020, 3, 0.37299975634739685]],
 102.06: [[2000.0, 0.0, 0.0],
  [2001.0, 0.0, 0.0],
  [2002.0, 0.0, 0.0],
  [2003.0, 13.0, 4.39],
  [2004.0, 0.0, 0.0],
  [2005.0, 0.0, 0.0],
  [2006.0, 0.0, 0.0],
  [2007.0, 0.0, 0.0],
  [2008.0, 0.0, 0.0],
  [2009.0, 0.0, 0.0],
  [2010.0, 0.0, 0.0],
  [2011.0, 0.0, 0.0],
  [2012.0, 0.0, 0.0],
  [2013.0, 0.0, 0.0],
  [2014.0, 7.0, 2.26],
  [2015.0, 6.0, 1.89],
  [2016.0, 4.0, 1.23]],
 102.08: [[2000.0, 0.0, 0.0],
  [2001.0, 0.0, 0.0],
  [2002.0, 0.0, 0.0],
  [2003.0, 8.0, 3